In [1]:
# import all the things
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

## Extract Data

In [2]:
# path to Golden Globes csv file
gg_file = "Resources/golden_globe_awards.csv"

# read in Golden Globes csv file
gg_df_orig = pd.read_csv(gg_file)

# drop unnecessary columns ceremony and year_film
gg_df = gg_df_orig.drop(columns={"ceremony"})
gg_df.head()

,year_film,year_award,category,nominee,film,win
0,1943,1944,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


In [3]:
# path to Movies csv file
movies_file = "Resources/AllMoviesDetailsCleaned.csv"

# read in Movies csv file
movies_df_orig = pd.read_csv(movies_file, sep=';')

# Delete columns that are not needed
movies_df = movies_df_orig.drop(columns={'id', 'imdb_id', 'original_language', 'overview', 'production_companies',
       'production_countries', 'runtime', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count',
       'production_companies_number', 'production_countries_number', 'spoken_languages_number', 'original_title'})

movies_df.head()

,budget,genres,popularity,release_date,revenue,title
0,0,Drama|Crime,0.823904,21/10/1988,0,Ariel
1,0,Drama|Comedy,0.47445,16/10/1986,0,Shadows in Paradise
2,4000000,Crime|Comedy,1.698,25/12/1995,4300000,Four Rooms
3,0,Action|Thriller|Crime,1.32287,15/10/1993,12136938,Judgment Night
4,42000,Documentary,0.054716,01/01/2006,0,Life in Loops (A Megacities RMX)


## Transform Data

### Golden Globes

In [4]:
# remove non movie based categories
gg_movies_df = gg_df[gg_df['category'].str.contains('Actor') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Actress') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Director') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('International') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Special') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Screenplay') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Song') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Score') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Cinematography') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Foreign') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Star') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Juvenile') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('International') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Color') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Henrietta') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Cecil') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('International') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Documentary') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Television') == False]
gg_movies_df = gg_movies_df[gg_movies_df['category'].str.contains('Hollywood') == False]

# drop film column and rename nominee column to film
gg_movies_df = gg_movies_df.drop(columns="film")
gg_movies_df = gg_movies_df.rename(columns={"nominee":"film"})

#keep movies before Jan 2017
gg_movies_df = gg_movies_df[gg_movies_df['year_film']<=2016]

#keep movies after 1997
gg_movies_df = gg_movies_df[gg_movies_df['year_film']>= 1997]

# read the movie title list into a column for cleaning
movie_title_list = gg_movies_df["film"].tolist()

# remove any "The" at the start of the movie name
# remove any ", The" at the end of the movie name
for i in range(0, len(movie_title_list)):
    if movie_title_list[i][-5:] == ", The":
        movie_title_list[i] = movie_title_list[i][:-5]
    if movie_title_list[i][:4] == "The ":
        movie_title_list[i] = movie_title_list[i][4:]
    if movie_title_list[i][-3:] == ", A":
        movie_title_list[i] = movie_title_list[i][:-3]
    if movie_title_list[i][-4:] == ", An":
        movie_title_list[i] = movie_title_list[i][:-4]
    movie_title_list[i] = movie_title_list[i].lower()    

# rename film to old film title
gg_movies_df = gg_movies_df.rename(columns={"film":"old_film_title"})

# add cleaned film title to data frame
gg_movies_df["film"] = movie_title_list

#clean for characters and spaces
gg_movies_df["film"] = gg_movies_df["film"].str.replace(".", "")
gg_movies_df["film"] = gg_movies_df["film"].str.replace("-", "")
gg_movies_df["film"] = gg_movies_df["film"].str.replace(" ", "")
gg_movies_df["film"] = gg_movies_df["film"].str.replace("·", "")

gg_movies_df

,year_film,year_award,category,old_film_title,win,film
5082,1997,1998,Best Motion Picture - Drama,Titanic,True,titanic
5083,1997,1998,Best Motion Picture - Drama,"Boxer, The",False,boxer
5084,1997,1998,Best Motion Picture - Drama,Good Will Hunting,False,goodwillhunting
5085,1997,1998,Best Motion Picture - Drama,L.A. Confidential,False,laconfidential
5086,1997,1998,Best Motion Picture - Drama,Amistad,False,amistad
...,...,...,...,...,...,...
7534,2016,2017,Best Motion Picture - Animated,Zootopia,True,zootopia
7535,2016,2017,Best Motion Picture - Animated,Kubo and the Two Strings,False,kuboandthetwostrings
7536,2016,2017,Best Motion Picture - Animated,Moana,False,moana
7537,2016,2017,Best Motion Picture - Animated,My Life as a Zucchini,False,mylifeasazucchini


### Movie List

In [5]:
#to run this cell again, make sure to run cell #3 first

# Delete row with NaN in the title column
movies_df.dropna(subset=['title'], inplace = True)

movies_df = movies_df.rename(columns={"title":"old_title"})
movies_df['title'] = movies_df['old_title'].str.lower()
movies_df['release_year'] = pd.DatetimeIndex(movies_df['release_date']).year.astype('Int64')
movies_df['budget'] = movies_df['budget'].replace(0,np.NaN)    
movies_df['profit'] = movies_df['revenue'] - movies_df['budget']

# read the movie title list into a column for cleaning
movies_df_title_list = movies_df["title"].tolist()

for i in range(0, len(movies_df_title_list)):
    if movies_df_title_list[i][-5:] == ", the":
        movies_df_title_list[i] = movies_df_title_list[i][:-5]
    if movies_df_title_list[i][:4] == "the ":
        movies_df_title_list[i] = movies_df_title_list[i][4:]

movies_df["title"] = movies_df_title_list

#clean for characters and spaces
movies_df["title"] = movies_df["title"].str.replace(".", "")
movies_df["title"] = movies_df["title"].str.replace("-", "")
movies_df["title"] = movies_df["title"].str.replace(" ", "")
movies_df["title"] = movies_df["title"].str.replace("·", "")

#keep movies before Jan 2017
movies_df = movies_df[movies_df['release_year']<=2016]

#keep movies after 1997
movies_df = movies_df[movies_df['release_year']>= 1997]

movies_df

#to run this cell again, make sure to run cell #3 first

,budget,genres,popularity,release_date,revenue,old_title,title,release_year,profit
4,42000.0,Documentary,0.054716,01/01/2006,0,Life in Loops (A Megacities RMX),lifeinloops(amegacitiesrmx),2006,-42000.0
5,NaN,Drama,0.001647,02/09/2004,0,Sunday in August,sundayinaugust,2004,NaN
7,94000000.0,Animation|Family,9.915573,30/05/2003,940335536,Finding Nemo,findingnemo,2003,846335536.0
9,15000000.0,Drama,8.191009,15/09/1999,356296601,American Beauty,americanbeauty,1999,341296601.0
11,12800000.0,Drama|Crime|Music,2.106217,17/05/2000,40031879,Dancer in the Dark,dancerinthedark,2000,27231879.0
...,...,...,...,...,...,...,...,...,...
329030,NaN,Horror,0.1503,06/11/2004,0,The Inn,inn,2004,NaN
329037,NaN,Comedy,0.0003,08/03/2003,0,The Tom Green Show: Early Exposure - Raw Meat ...,tomgreenshow:earlyexposurerawmeatandraretreats,2003,NaN
329039,NaN,Documentary|History|TV Movie,0.0045,21/01/2016,0,The Day Hitler Died,dayhitlerdied,2016,NaN
329040,NaN,NaN,0.00045,24/11/2006,0,Mobb Deep - Life of the Infamous: The Videos,mobbdeeplifeoftheinfamous:thevideos,2006,NaN


## Merge

#### First Merge (1/3)

In [6]:
#FIRST merge
merge_df_1 = pd.merge(gg_movies_df, movies_df, how="outer", left_on=['film','year_film'], 
                        right_on = ['title','release_year'])

#store items that did not match in other data frames
gg_nan_df_1 = merge_df_1[merge_df_1['title'].isnull()]
gg_nan_df_1 = gg_nan_df_1[["year_film", "year_award", "category", "old_film_title", "win", "film"]]

movies_nan_df_1 = merge_df_1[merge_df_1['film'].isnull()]
movies_nan_df_1 = movies_nan_df_1[["budget", "genres", "popularity", "release_date", "revenue", "old_title", 
                              "title", "release_year", "profit"]]

#keep merged items
merge_df_1 = merge_df_1[merge_df_1['title'].isnull() == False]
merge_df_1 = merge_df_1[merge_df_1['film'].isnull() == False]
merge_df_1.head()

,year_film,year_award,category,old_film_title,win,film,budget,genres,popularity,release_date,revenue,old_title,title,release_year,profit
0,1997,1998.0,Best Motion Picture - Drama,Titanic,True,titanic,200000000.0,Drama|Romance|Thriller,10.517521,18/11/1997,1.845034e+09,Titanic,titanic,1997,1.645034e+09
1,1997,1998.0,Best Motion Picture - Drama,"Boxer, The",False,boxer,NaN,Drama|Romance,0.783047,31/12/1997,0.000000e+00,The Boxer,boxer,1997,NaN
2,1997,1998.0,Best Motion Picture - Drama,Good Will Hunting,False,goodwillhunting,10000000.0,Drama,5.439261,05/12/1997,2.259334e+08,Good Will Hunting,goodwillhunting,1997,2.159334e+08
3,1997,1998.0,Best Motion Picture - Drama,L.A. Confidential,False,laconfidential,35000000.0,Crime|Drama|Mystery|Thriller,2.984624,19/09/1997,1.262169e+08,L.A. Confidential,laconfidential,1997,9.121694e+07
4,1997,1998.0,Best Motion Picture - Drama,Amistad,False,amistad,36000000.0,Drama|History|Mystery,0.566813,03/12/1997,7.400000e+07,Amistad,amistad,1997,3.800000e+07


#### Second Merge of Two DataFrames (2/3)

In [7]:
#second merge correcting film or release year

#see if subtracting a year will help merge
gg_nan_df_1['year_film_minus'] = gg_nan_df_1['year_film'] -1

#SECOND: perform merge using df with non-matched rows from first merge
merge_df_2 = pd.merge(gg_nan_df_1, movies_nan_df_1, how="outer", left_on=['film','year_film_minus'], 
                        right_on = ['title','release_year'])

#drop column
merge_df_2 = merge_df_2.drop(columns={"year_film_minus"})

#store items that did not match in other data frames
gg_nan_df_2 = merge_df_2[merge_df_2['title'].isnull()]
gg_nan_df_2 = gg_nan_df_2[["year_film", "year_award", "category", "old_film_title", "win", "film"]]

movies_nan_df_2 = merge_df_2[merge_df_2['film'].isnull()]
movies_nan_df_2 = movies_nan_df_2[["budget", "genres", "popularity", "release_date", "revenue", "old_title", 
                              "title", "release_year", "profit"]]

#keep merged items
merge_df_2 = merge_df_2[merge_df_2['title'].isnull() == False]
merge_df_2 = merge_df_2[merge_df_2['film'].isnull() == False]
merge_df_2.head()

,year_film,year_award,category,old_film_title,win,film,budget,genres,popularity,release_date,revenue,old_title,title,release_year,profit
0,2000,2001.0,Best Motion Picture - Drama,Sunshine,False,sunshine,NaN,Drama|History,1.006455,13/09/1999,0.0,Sunshine,sunshine,1999,NaN
2,2003,2004.0,Best Motion Picture - Musical or Comedy,Bend It Like Beckham,False,benditlikebeckham,3500159.0,Comedy|Drama|Romance,2.425541,11/04/2002,76578641.0,Bend It Like Beckham,benditlikebeckham,2002,73078482.0
3,2006,2007.0,Best Motion Picture - Musical or Comedy,Thank You For Smoking,False,thankyouforsmoking,6500000.0,Comedy|Drama,1.890573,05/09/2005,24793509.0,Thank You for Smoking,thankyouforsmoking,2005,18293509.0
4,2009,2010.0,Best Motion Picture - Drama,"Hurt Locker, The",False,hurtlocker,15000000.0,Drama|Thriller|War,3.696055,10/10/2008,42000000.0,The Hurt Locker,hurtlocker,2008,27000000.0
8,2012,2013.0,Best Motion Picture - Musical or Comedy,"Best Exotic Marigold Hotel, The",False,bestexoticmarigoldhotel,10000000.0,Drama|Comedy,1.710304,28/11/2011,136836156.0,The Best Exotic Marigold Hotel,bestexoticmarigoldhotel,2011,126836156.0


#### Third Merge of Two DataFrames (3/3)

In [8]:
#third merge with individual changes

# zerodarkthirty\t'
gg_nan_df_2['film'] = gg_nan_df_2['film'].str.replace("\t", "")
# mulhollanddr
gg_nan_df_2['film'] = gg_nan_df_2['film'].str.replace("mulhollanddr", "mulhollanddrive")
# wreckitralph
gg_nan_df_2['film'] = gg_nan_df_2['film'].str.replace("wreckitralph", "bitbybitcreatingtheworldsofwreckitralph")
movies_nan_df_2['release_year'][movies_nan_df_2.title == 'bitbybitcreatingtheworldsofwreckitralph'] = 2012

# lesmisérables:arevolutionaryapproach
movies_nan_df_2['title'] = movies_nan_df_2['title'].str.replace("lesmisérables:arevolutionaryapproach", "lesmisérables")

# precious:basedonthenovelpushbysapphire
gg_nan_df_2['film'] = gg_nan_df_2['film'].str.replace("precious:basedonthenovelpushbysapphire", "precious")

# THIRD: perform merge using df with non-matched rows from second (and first) merge
merge_df_3 = pd.merge(gg_nan_df_2, movies_nan_df_2, how="outer", left_on=['film','year_film'], 
                        right_on = ['title','release_year'])

#store items that did not match in other data frames
gg_nan_df_3 = merge_df_3[merge_df_3['title'].isnull()]
gg_nan_df_3 = gg_nan_df_3[["year_film", "year_award", "category", "old_film_title", "win", "film"]]

movies_nan_df_3 = merge_df_3[merge_df_3['film'].isnull()]
movies_nan_df_3 = movies_nan_df_3[["budget", "genres", "popularity", "release_date", "revenue", "old_title", 
                              "title", "release_year", "profit"]]
# Note
# silverliningsplaybook not in movies db
# howtotrainyourdragon2 not in movies db
merge_df_3.head()

,year_film,year_award,category,old_film_title,win,film,budget,genres,popularity,release_date,revenue,old_title,title,release_year,profit
0,2001,2002.0,Best Motion Picture - Drama,Mulholland Dr.,False,mulhollanddrive,15000000.0,Thriller|Drama|Mystery,4.371764,16/05/2001,20117339.0,Mulholland Drive,mulhollanddrive,2001,5117339.0
1,2009,2010.0,Best Motion Picture - Drama,Precious: Based On The Novel Push By Sapphire,False,precious,10000000.0,Drama,1.999454,06/11/2009,47536959.0,Precious,precious,2009,37536959.0
2,2012,2013.0,Best Motion Picture - Drama,Zero Dark Thirty \t,False,zerodarkthirty,40000000.0,Thriller|Drama|History,4.552182,19/12/2012,132820716.0,Zero Dark Thirty,zerodarkthirty,2012,92820716.0
3,2012,2013.0,Best Motion Picture - Musical or Comedy,Les Misérables,True,lesmisérables,NaN,Documentary,0.001006,18/02/2012,0.0,Les Misérables: A Revolutionary Approach,lesmisérables,2012,NaN
4,2012,2013.0,Best Motion Picture - Musical or Comedy,Silver Linings Playbook,False,silverliningsplaybook,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Clean Final DataFrame

In [9]:
#fully merged dataframe
gg_movie_fullmerge = merge_df_1.append(merge_df_2)
gg_movie_fullmerge = gg_movie_fullmerge.append(merge_df_3)

#gg df rows that did not match to the movie df
gg_unmerged = gg_nan_df_3

#movie df rows that did not match to gg df
movie_unmerged = movies_nan_df_3

gg_movie_final = gg_movie_fullmerge[["old_title", "genres", "release_year", "category", "year_award", "win",
                                    "popularity", "budget", "revenue", "profit"]]
gg_movie_final.rename(columns = {"old_title": "film_title",
                                 "category": "nom_category",
                                 "year_award": "nom_year"})
gg_movie_final

,old_title,genres,release_year,category,year_award,win,popularity,budget,revenue,profit
0,Titanic,Drama|Romance|Thriller,1997,Best Motion Picture - Drama,1998.0,True,10.517521,200000000.0,1.845034e+09,1.645034e+09
1,The Boxer,Drama|Romance,1997,Best Motion Picture - Drama,1998.0,False,0.783047,NaN,0.000000e+00,NaN
2,Good Will Hunting,Drama,1997,Best Motion Picture - Drama,1998.0,False,5.439261,10000000.0,2.259334e+08,2.159334e+08
3,L.A. Confidential,Crime|Drama|Mystery|Thriller,1997,Best Motion Picture - Drama,1998.0,False,2.984624,35000000.0,1.262169e+08,9.121694e+07
4,Amistad,Drama|History|Mystery,1997,Best Motion Picture - Drama,1998.0,False,0.566813,36000000.0,7.400000e+07,3.800000e+07
...,...,...,...,...,...,...,...,...,...,...
163091,The Inn,Horror,2004,NaN,NaN,NaN,0.1503,NaN,0.000000e+00,NaN
163092,The Tom Green Show: Early Exposure - Raw Meat ...,Comedy,2003,NaN,NaN,NaN,0.0003,NaN,0.000000e+00,NaN
163093,The Day Hitler Died,Documentary|History|TV Movie,2016,NaN,NaN,NaN,0.0045,NaN,0.000000e+00,NaN
163094,Mobb Deep - Life of the Infamous: The Videos,NaN,2006,NaN,NaN,NaN,0.00045,NaN,0.000000e+00,NaN


## Load

In [10]:

# connection_string = "postgres:postgres@localhost:5432/customer_db"
# engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Confirm tables
# engine.table_names()

In [12]:
# premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

# county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)